# WSU size of computing estimate numbers (Database)

The goal here is to put together some numbers on the type of projects ALMA would process as part of the WSU to be used to produce a total size of computing estimate.

Amanda Kepley (20220921)

In [1]:
import numpy as np
import astropy.units as u
from ast import literal_eval
from astropy import constants as const
from matplotlib import pyplot as plt, ticker as mticker
import re
import math
from astropy.table import Table, QTable, vstack
from astropy import constants as const
from importlib import reload

## TOC

* [Read in Data](#readin)
* [Generate WSU DB](#wsu_db)

## Read in massaged cycle 7 and 8 data <a id="readin"></a>

In [2]:
cycle7tab = Table.read('result_table_cycle7_with_calc_values_20220923.csv')
cycle8tab = Table.read('result_table_cycle8_with_calc_values_20220923.csv')

Information that we already have:
* FOV
* resolution
* mosaic or not
* frequency
* image linear size
* number of baseline (7m vs. 12m)
* n polarizations

Information that I need to calculate for WSU:
* Bandwidth (16GHz for all receivers)
* channel width (follow procedure in data memo)
* nchan (calculate from bandwidth and channel width)
* dump time (is this just a 7m vs. 12m dichotomy). Looks like in her spreadsheet Crystal uses 3.024s for 12m and 10.08s for 7m.

The data rate memo calculated the number of channels two ways:
* finest spectral resolution across all spw for cycle 7 project [the writeup says sbs, but I think they mean spws. confirm with crystal?]
* fix resolution:
  * *> 10km/s* -> 10km/s
  * 1-10 km/s -> 1km/s
  * 0.1-1 km/s -> 0.1km/s
  * <0.1km/s -> 0.1km/s

How am I going to get to a fraction of projects? Using the time estimate doesn't seem right because the time for spectral scans will be drastically faster. What I can do is just count the number of MOUS'es and divide by the total.

Another question is what to do for projects with multiple targets? Just do the same thing for all targets. That might make sense since you usually want the same spectral setup for all your sources. But you might have a different mosaic size. Can I take largest? or average? That might be easiest.


## Put together WSU mous data base <a id="wsu_db"></a>

In [3]:
import wsu_db

In [4]:
reload(wsu_db)

<module 'wsu_db' from '/Users/akepley/Dropbox/Support/naasc/mitigation/wsu_db.py'>

In [5]:
result = wsu_db.create_database(cycle7tab)

In [6]:
result_c8 = wsu_db.create_database(cycle8tab)

In [7]:
# save data bases if what's desired
result.write('cycle7wsu_20221003.fits',overwrite=True)
result_c8.write('cycle8wsu_20221003.fits',overwrite=True)

In [8]:
result.columns

<TableColumns names=('mous','proposal_id','array','nant_typical','nant_array','nant_all','band','ntarget','s_fov','s_resolution','mosaic','imsize','pb','cell','npol','velocity_resolution_current','wsu_freq','wsu_bandwidth_initial','wsu_bandwidth_final','wsu_bandwidth_spw','wsu_specwidth_finest','wsu_chanavg_finest','wsu_specwidth_stepped','wsu_chanavg_stepped','wsu_specwidth_stepped2','wsu_chanavg_stepped2','tint','nbase_typical','nbase_array','nbase_all','wsu_nchan_final_finest','wsu_nchan_final_stepped','wsu_nchan_final_stepped2','wsu_nchan_spw_finest','wsu_nchan_spw_stepped','wsu_nchan_spw_stepped2','wsu_nchan_initial_finest','wsu_nchan_initial_stepped','wsu_nchan_initial_stepped2','vis_rate_typical_initial_finest','vis_rate_typical_initial_stepped','vis_rate_typical_final_finest','vis_rate_typical_final_stepped','vis_rate_typical_final_stepped2','vis_rate_array_initial_finest','vis_rate_array_initial_stepped','vis_rate_array_final_finest','vis_rate_array_final_stepped','vis_rate_ar

In [9]:
len(result)

11519

In [ ]:
## Mosaic imsize investigation

In [ ]:
idx = (result['mosaic'] == 'T') & (result['imsize'] >5800)
result['mous','imsize','cell','s_fov','s_resolution','wsu_freq','pb','mosaic'][idx]

In [ ]:
0.01056278408537675 *3600.0  

Image pre-check values for  2019.1.00796.S, uid://A001/X1471/X317	

* beam = 0.0457 x 0.0404 arcsec
* cell = 0.0081 x 0.0081 arcsec

Unmitigated imsize calculated in pipeline for X317 is 7776, 7776 according to SCG tests

eye balling the spatial set up it looks like there's 10-12arcsec between pointings and the plot says the primary beam is 26.0arcsec

The pipeline math is   

npts <= 3
* nxpix = int((1.65 * beam_radius_v + xspread) / cellx_v)

npts >3
* nxpix = int((1.5 * beam_radius_v + xspread) / cellx_v)

We only have two pointings here.



In [ ]:
(26.0 + 10.0)

In [ ]:
(1.65 * 26.0 + 10.0)/0.0081

So my estimate is a little on the low end, but not crazy

In [ ]:
(0.01044*3600+25.6*0.70)/0.0072

## Imsize investigation

Something is  odd with my image sizes. I'm using 2019.1.01463.S uid://A001/X1465/Xc05 as my poster child

For the unmitigated imaging done by the pipeline, the pipeline calculates the following values:
* beam: 0.0322" x 0.0211"
* cell: 0.0042" x 0.0042"
* imsize: [11250, 11250] pixels
* FOV: 47.25 arcsec

Now let's look at what I get from my calculations

In [ ]:
#2019.1.01463.S
idx =result['mous'] == 'uid://A001/X1465/Xc05'
result['mous','s_fov','s_resolution','imsize','wsu_nchan_final_stepped','wsu_nchan_final_finest','mosaic'][idx]

In [ ]:
np.log10(237037.03703703705)

In [ ]:
np.log10(32921.81069958848)

In [ ]:
# the imsize is
0.007157768473981626*3600.00 # arcsec

In [ ]:
# What's the estimated imsize at this frequency??
# frequency
freq = 218.821 #GHz
19.4*300/218.821

This is comparable to the imsize calculated above.

In [ ]:
# What pixel size does this imply for five pixels per beam?
0.024588/5.0

In [ ]:
# What pixel size does this imply for six pixels per beam?
0.024588/6.0

What happens if I use the points_per_fov value??

In [ ]:
idx2 = cycle7tab['member_ous_uid'] == 'uid://A001/X1465/Xc05'
cycle7tab['proposal_id','member_ous_uid','s_fov','s_resolution','points_per_fov','spw_nchan','is_mosaic'][idx2]

In [ ]:
# imsize from points per fov value
np.sqrt(1100957.4775723005)*5.0

Matches imsize above.

So it looks like the FOV is the difference:

In [ ]:
(47.25/25.76)*5250

Still an underestimate, but closer.

The pipeline calculates the primary beam as

primary_beam_size = \
            1.22 \
            * cqa.getvalue(cqa.convert(cqa.constants('c'), 'm/s')) \
            / ref_frequency \
            / smallest_diameter \
            * (180.0 * 3600.0 / math.pi)

In [ ]:
1.22 * ((const.c.value /  218.821e9) / (12.0) )*(180*3600.0/math.pi)

Pipeline calculation is here:

beam_radius_v = primary_beam

beam_fwhp = 1.12 / 1.22 * beam_radius_v

nxpix = int(utils.round_half_up(1.1 * beam_fwhp * math.sqrt(-math.log(sfpblimit) / math.log(2.)) / cellx_v))

In [ ]:
(1.12/1.22)* 28.73

In [ ]:
1.1 * 26.38 * math.sqrt(-math.log(0.2) / math.log(2.0))

Okay. This is the value I get above. 

What's the constant??

In [ ]:
1.1* (1.12/1.22)*math.sqrt(-math.log(0.2) / math.log(2.0))

In [ ]:
1.54*25.8

In [ ]:
40.0/0.0040